<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/21June_GeneticAlgorithm_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import numba
from plotly.subplots import make_subplots
np.set_printoptions(formatter={'float':lambda x: f'{x:0.2f}'})

In [61]:
rng=np.random.RandomState(123)

In [112]:
N=100
coords=rng.uniform(0,10, size=(N,2))

In [113]:
dist=np.sqrt(np.sum((coords[:,None,:]-coords[None,:,:])**2,axis=2))

In [114]:
def calcdist(seq):
    return np.sum(dist[seq[1:,:],seq[:-1,:]]) + dist[seq[0,:],seq[-1,:]]

In [115]:
def topn(seq,n):
    dists=calcdist(seq)
    topn_idx=np.argsort(dists)[:n]
    return seq[:,topn_idx], dists[topn_idx]

In [116]:
popsize = 512
seq=np.zeros((N,popsize), dtype=int)

for popidx in range(popsize):

    remain = list(range(N))

    cur = rng.randint(N)
    seq[0,popidx]=cur
    remain.remove(cur)
    for i in range(1,N-1):
        best2= np.argsort(dist[cur,remain])[:2]
        best=remain[best2[rng.choice([0,1],p=(0.95,0.05))]  ]
        seq[i,popidx]=best
        cur=best
        remain.remove(best)
    seq[-1,popidx]=remain[0]


In [117]:
top20, dists20 = topn(seq,20)

In [118]:
def plotter(seq_plot, dists, nr, nc):
    fig=make_subplots(rows = nr, cols=nc)
    seq = np.r_[seq_plot, np.atleast_2d(seq_plot[0,:])]
    for i in range(seq.shape[1]):
        fig.add_scatter(x=coords[seq[:,i],0], y=coords[seq[:,i],1], row=i//nc+1, col=i%nc +1)
    fig.update_layout(template='plotly_dark', showlegend=False)
    fig.show()

In [119]:
plotter(top20, dists20, nr=4, nc=5)

In [131]:
@numba.njit
def isin(val, arr):
    for item in arr:
        if val==item:
            return True
    return False

@numba.njit
def swap(p1,p2,c1,c2):
    o=np.zeros_like(p1)
    N=p1.size
    swap=np.concatenate((p2[c2:],p2[:c2]))
    retain=p1[c1:c2]

    filtered=[]
    for i in swap:
        if not(isin(i,retain)):
            filtered.append(i)
    o[c1:c2]=p1[c1:c2]
    o[c2:]=filtered[:(N-c2)]
    o[:c1]=filtered[(N-c2):]
    return o

@numba.njit
def mate2p(p1,p2):
    N=p1.size
    c1=np.random.randint(N//4,N//2)
    c2=np.random.randint(N//2, 3*N//4)
    
    o1=swap(p1,p2,c1,c2)
    o2=swap(p2,p1,c1,c2)
    return o1,o2

@numba.njit
def mate(s):
    seq=s[:,np.random.permutation(np.arange(s.shape[1]))]
    offspring=np.zeros_like(seq)
    for i,(p1,p2) in enumerate(zip(seq[:,::2].T, seq[:,1::2].T)):
        offspring[:,2*i],offspring[:,2*i+1]=mate2p(p1,p2)
    return offspring

In [133]:
%%timeit
mate(seq)

100 loops, best of 5: 3.3 ms per loop


In [110]:
144
36
5.53
3.36
3.3

array([[8, 8, 4, 5, 1, 9, 3, 3, 7, 8, 1, 2, 8, 5, 5, 1, 7, 9, 7, 5],
       [6, 6, 9, 7, 5, 4, 9, 2, 5, 6, 5, 0, 6, 7, 7, 5, 5, 4, 5, 7],
       [4, 4, 2, 9, 7, 0, 4, 0, 1, 4, 7, 4, 4, 9, 9, 7, 1, 0, 1, 9],
       [9, 9, 0, 4, 9, 2, 0, 4, 4, 9, 9, 9, 9, 4, 4, 9, 4, 2, 4, 4],
       [2, 2, 6, 0, 4, 3, 2, 9, 9, 2, 4, 7, 2, 0, 0, 4, 9, 3, 9, 0],
       [0, 0, 8, 2, 0, 7, 7, 7, 2, 0, 0, 5, 3, 2, 2, 0, 2, 7, 2, 2],
       [3, 3, 1, 3, 2, 5, 5, 5, 0, 3, 2, 1, 0, 3, 3, 2, 0, 5, 0, 3],
       [7, 7, 5, 1, 3, 1, 1, 1, 6, 7, 3, 8, 1, 1, 1, 3, 6, 1, 6, 1],
       [5, 5, 7, 8, 6, 8, 8, 8, 8, 5, 6, 6, 5, 8, 8, 6, 8, 8, 8, 8],
       [1, 1, 3, 6, 8, 6, 6, 6, 3, 1, 8, 3, 7, 6, 6, 8, 3, 6, 3, 6]])

In [84]:
for i, (vehicle,fruit) in enumerate(zip(a,b)):
    print(f'Step {i}: I ate a {fruit} in a {vehicle}')

Step 0: I ate a apple in a car
Step 1: I ate a orange in a boat
Step 2: I ate a banana in a jet


In [83]:
for i, fruit in enumerate(b):
    print(i, fruit)

0 apple
1 orange
2 banana


In [47]:
p1=np.array([3,4,8,2,7,1,6,5])
p2=np.array([4,2,5,1,6,8,3,7])

In [43]:
arr=np.arange(int(1e7))

In [50]:
%%timeit
isin(int(9.99e6),arr)

100 loops, best of 5: 9.58 ms per loop
